<a href="https://colab.research.google.com/github/DuyDucNguyen/Practical_FEM_FEniCS_Colab/blob/master/dual_based_error_estimate_Poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dual-based error estimates

In [0]:
# This program shows Average Deflection Goal Function is a good quality to measure U. Apply to Poisson equation.

# Reference: This program is followed the ideas in last chapter of the course 'edx.org - KTHx - High Performance Finite Element Modeling'. 
# The author modifies it up to date and give clear explaination.  

# Copyright (C) 2019 Duy Duc NGUYEN (duyduc.nguyen@protonmail.com)

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2019-09-23
# Last changed: 2019-09-23

# This demo is maintained by Duy Duc NGUYEN
# Please report possible problems to duyduc.nguyen@protonmail.com

In [0]:
# install Fenics: 2 mins
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

**Primal problem**: Consider Poisson's equation

\begin{equation}
\left\{ \begin{array}{ccc}
-\Delta u=f & \text{ in } & \Omega\\
u=0 & \text{ on } & \partial\Omega
\end{array}\right.
\end{equation}

where $\Omega=[0,1]\times[0,1]$, the source

\begin{equation}
f=32x_{1}\left(1-x_{1}\right)+32x_{2}\left(1-x_{2}\right)
\end{equation}

the exact solution to this problem is 
\begin{equation}
u_{e}=16x_{1}\left(1-x_{1}\right)x_{2}\left(1-x_{2}\right)
\end{equation}

residual 
\begin{align*}
r\left(u,v\right) & =a\left(u,v\right)-L\left(v\right)=0,\\
a\left(u,v\right) & =\int_{\Omega}\nabla u\nabla v\ \mathrm{d}x\\
L\left(v\right) & =\int_{\Omega}fv\ \mathrm{d}x
\end{align*}

The following code finds a linear approximation to the solution 

**Goal function:** Consider Average deflection 

\begin{equation}
\mathcal{M}(u)=\int_{\Omega}u\ \mathrm{d}x
\end{equation}


**Dual problem:** find $\phi$ such that
\begin{equation}
a\left(v,\phi\right)=\mathcal{M}\left(v\right)
\end{equation}
or 
\begin{equation}
\int_{\Omega}\nabla v\nabla\phi\ \mathrm{d}x=\int_{\Omega}1v\ \mathrm{d}x
\end{equation}
Apply the Green rule to the bilinear operator
\begin{equation}
-\int_{\Omega}v\nabla\nabla\phi\ \mathrm{d}x=\int_{\Omega}1.v\ \mathrm{d}x
\end{equation}
Strong form: The dual problem to Poisson's equation is defined on
the same domain with homogeneous DC

\begin{equation}
\left\{ \begin{array}{ccc}
-\Delta\phi=\psi & \text{ in } & \Omega\\
\phi=0 & \text{ on } & \partial\Omega
\end{array}\right.
\end{equation}
with the right hand side $\psi=1$, a piecewise quadratic finite element
approximation to the dual problem can found as follows:

we now study the relationship between two different estimates of the
error of the approximate solution $u_{h}$. We define $e_{1}$ and
$e_{2}$ as follows: 
\begin{align*}
e_{1} & =\mathcal{M}\left(e\right)=\mathcal{M}\left(u_{h}-u_e\right)=\int_{\Omega}\left(u_{h}-u_{e}\right)\psi\ \mathrm{d}x\\
e_{2} & =r\left(u_{h},\phi_h\right)=\int_{\Omega}\nabla u_{h}\cdot\nabla\phi_h-f\phi_h\ \mathrm{d}x
\end{align*}


In [2]:
from mshr import *; 
import time;
import numpy as np
import matplotlib.pyplot as plt


def dual_based_error(resolution):
	# create mesh
	mesh = UnitSquareMesh(resolution, resolution)

		
	def AllBoundary(x, on_boundary):
	    return on_boundary

	zero = Constant(0.0)

	def aform(u, v):
	    return inner(grad(u), grad(v))

	def Lform(f, v):
	    return f*v

	# Primal problem
	f  = Expression("32.*x[0]*(1. - x[0])+32.*x[1]*(1. - x[1])", domain=mesh, degree=5)
	ue = Expression("16.*x[0]*(1. - x[0])*x[1]*(1. - x[1])", domain=mesh, degree=5)

	Qp  = FunctionSpace(mesh,'CG',1);
	bcp = DirichletBC(Qp, zero, AllBoundary)

	u = TrialFunction(Qp);
	v = TestFunction(Qp);

	ap = aform(u, v)*dx 
	Lp = Lform(f, v)*dx 

	U = Function(Qp)
	solve(ap == Lp, U, bcp)


	# Dual problem
	# CG: Continuous Galerkin
	Qd  = FunctionSpace(mesh, 'CG', 2); 
	psi = Constant(1.0)
	bcd = DirichletBC(Qd, zero, AllBoundary)

	w   = TestFunction(Qd);
	phi = TrialFunction(Qd);
	ad  = aform(w, phi)*dx 
	Ld  = Lform(psi, w)*dx
	phi = Function(Qd)
	solve(ad == Ld, phi, bcd)


	# Compute errors
	# DG: Discontinuous Galerkin
	Z = FunctionSpace(mesh, "DG", 0); 
	z = TestFunction(Z)
	ef1 = Function(Z)
	ef2 = Function(Z)


	# error from M
	ef1.vector()[:] = assemble( (U-ue)*psi*z*dx )[:]
	ef1.vector()[:] = abs(ef1.vector()[:])
		
		
	# error computed residual
	ef2.vector()[:] = assemble( ( aform(U, phi) - Lform(f, phi) )*z*dx )[:]
	ef2.vector()[:] = abs(ef2.vector()[:])


	# ==== Plots ====
	#plt.figure()
	#p1 = plot(ef1)
	#plt.colorbar(p1)

	#plt.figure()
	#p2 = plot(ef2)
	#plt.colorbar(p2)
		
	e1 = abs( assemble( (U-ue)*psi*dx ) )
	e2 = abs( assemble( (inner(grad(U), grad(phi))-f*phi)*dx ) )
	print('e1:{}   e2:{}'.format(e1, e2))

	return e1, e2


if __name__ == "__main__": 
	dual_based_error(10)
	dual_based_error(20)
	dual_based_error(30)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FF

**Conclusion:** 

$e_1$ is not always comuted since $u_e$ is not always known. 

$e_2$ is computable and almost the same as $e_1$, so that, it is a good quality to measure how good $u_h$ is.